<a href="https://colab.research.google.com/github/zunaed-farabe/MH-project/blob/main/Meta_Heuristic_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.sparse import vstack, hstack
from scipy.sparse.linalg import svds
import random

def learn_low_rank_representation(X, s):
    n_samples, n_features = X.shape
    # Random sampling
    random_indices = random.sample(range(n_samples), s)
    C = X[random_indices, :]
    R = vstack([X, C])
    # SVD decomposition
    U, sigma, Vt = svds(R, k=s)
    # Extracting low-rank representation and residual error
    W = U[:n_samples, :]

    print("W.shape:", W.shape)
    print("Vt.T.shape:", Vt.shape)
    print("==================\n")
    residual_error = X - W.dot(Vt)
    return W, residual_error

def objective_function(x, W, residual_error):
    x = np.atleast_2d(x)
    solution = W.dot(x.T)
    residual_error_new = X - solution
    cost = np.sum(np.square(residual_error_new))
    return cost

def pso(x0, bounds, W, residual_error, n_generations, population_size, n_neighbors, phi1=2.05, phi2=2.05):
    n_variables = x0.shape[0]
    n_objs = 1
    population = np.random.uniform(bounds[:, 0], bounds[:, 1], (population_size, n_variables))
    velocity = np.zeros((population_size, n_variables))
    personal_best = population.copy()
    global_best = personal_best[np.argmin(objective_function(personal_best, W, residual_error))]
    pbest_values = objective_function(personal_best, W, residual_error)
    gbest_value = objective_function(global_best, W, residual_error)
    # Evolutionary computation
    for generation in range(n_generations):
        for i in range(population_size):
            velocity[i, :] = phi1 * np.random.rand() * (personal_best[i, :] - population[i, :]) + \
                              phi2 * np.random.rand() * (global_best - population[i, :])
            population[i, :] += velocity[i, :]
            # Constraint handling
            population[i, :] = np.clip(population[i, :], bounds[:, 0], bounds[:, 1])
            # Update personal and global bests
            obj_value = objective_function(population[i, :], W, residual_error)
            if obj_value < pbest_values[i]:
                personal_best[i, :] = population[i, :]
                pbest_values[i] = obj_value
                if obj_value < gbest_value:
                    global_best = population[i, :]
                    gbest_value = obj_value
        # Local version PSO
        sorted_indices = np.argsort(pbest_values)
        neighbors = sorted_indices[:n_neighbors]
        n_dimensions = x0.shape[0]
        neighbors_matrix = np.zeros((n_neighbors, n_dimensions))
        for i in range(n_neighbors):
            neighbors_matrix[i, :] = personal_best[neighbors[i], :]
        neighbors_mean = np.mean(neighbors_matrix, axis=0)
        # Mutation
        x0 = neighbors_mean + np.random.randn(n_variables) * np.std(neighbors_matrix, axis=0)
    return global_best

X = np.random.rand(100, 100)
W, residual_error = learn_low_rank_representation(X, 10)
n_variables = X.shape[1]
bounds = np.vstack([np.zeros])

W.shape: (100, 10)
Vt.T.shape: (10, 100)



In [6]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.5 MB/s eta 0:00:00


In [15]:
import numpy as np
from scipy.sparse.linalg import svds
from deap import base, creator, tools, algorithms


def learn_low_rank_representation(X, s=10):
    m, n = X.shape
    C = X[np.random.choice(m, size=s, replace=False), :]
    R = X[:, np.random.choice(n, size=C.shape[1], replace=False)]
    C = np.hstack((C, np.ones((s, 1))))
    R = np.hstack((np.ones((R.shape[0], 1)), R))
    U, sigma, Vt = svds(np.dot(C, R), k=s)
    W = np.dot(U[:, :s], Vt[:s, :])
    residual_error = np.linalg.norm(X - np.dot(W, np.dot(W.T, X)))
    return W, residual_error

def evolutionary_computation(W, residual_error, pop_size=100, num_generations=100):
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    def evaluate(individual):
        return np.linalg.norm(np.dot(W, individual) - X),

    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, -1, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=W.shape[1])
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate)

    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=num_generations, stats=stats, halloffame=hof, verbose=True)
    xT = hof[0]
    return xT

# Example usage
X = np.random.rand(100, 50)  # Replace with your dataset
W, residual_error = learn_low_rank_representation(X)
xT = evolutionary_computation(W, residual_error)
print("Final solution:", xT)

ValueError: shapes (10,51) and (100,51) not aligned: 51 (dim 1) != 100 (dim 0)

In [12]:
!pip install scipy

In [18]:
import numpy as np
from scipy.sparse import linalg
import random

def learn_low_rank_representation(X, s):
    # Randomly sample s rows/columns from X
    row_indices = random.sample(range(X.shape[0]), s)
    col_indices = random.sample(range(X.shape[1]), s)
    C = X[row_indices, :]
    R = X[:, col_indices]

    # Compute SVD of the concatenated matrices C and R
    _, _, VT = linalg.svds(np.vstack((C, R)), k=s)

    # Extract the first s right singular vectors and corresponding left singular vectors
    W = VT[:s, :].T

    # Calculate the residual error
    residual_error = X - W @ W.T @ X

    return W, residual_error

def evolutionary_computation(W, residual_error, num_generations, population_size):
    # Initialize the population around the attention subspace W
    x0 = np.random.normal(loc=W.flatten(), scale=0.1, size=(population_size, W.size))

    # Perform evolutionary computation using GA
    for generation in range(num_generations):
        # Evaluate fitness for each individual in the population
        fitness_scores = np.sum((x0.reshape(-1, W.shape[0], W.shape[1]) @ W.T @ residual_error)**2, axis=(1, 2))

        # Select parents for reproduction
        parent_indices = np.random.choice(population_size, size=population_size, replace=True, p=fitness_scores/np.sum(fitness_scores))
        parents = x0[parent_indices]

        # Perform crossover and mutation to create offspring
        offspring = np.empty_like(parents)
        for i in range(0, population_size, 2):
            parent1, parent2 = parents[i], parents[i+1]
            crossover_point = np.random.randint(1, W.size)
            offspring[i] = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            offspring[i+1] = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        mutation_mask = np.random.rand(population_size, W.size) < 0.1
        offspring[mutation_mask] += np.random.normal(scale=0.1, size=offspring[mutation_mask].shape)

        # Replace the population with the offspring
        x0 = offspring

    # Obtain the final solution
    xT = x0[np.argmin(np.sum((x0.reshape(-1, W.shape[0], W.shape[1]) @ W.T @ residual_error)**2, axis=(1, 2)))]

    return xT

# Example usage
X = np.random.rand(10, 10)  # Replace with your dataset
s = 3  # Number of rows/columns to sample
num_generations = 5  # Number of generations for evolutionary computation
population_size = 50  # Size of the population for GA

W, residual_error = learn_low_rank_representation(X, s)
xT = evolutionary_computation(W, residual_error, num_generations, population_size)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 10 and the array at index 1 has size 3

In [132]:

#chatGpt

import numpy as np
from scipy.sparse.linalg import svds
from deap import algorithms, base, creator, tools
global W
def learn_low_rank_representation(X, s):
    """
    Learn a low-rank representation of the dataset X.

    Parameters:
        X (numpy.ndarray): The dataset.
        s (int): The number of rows/columns to sample.

    Returns:
        W (numpy.ndarray): The low-rank representation.
        residual_error (numpy.ndarray): The residual error.
    """
    # Random sampling of s rows/columns
    indices = np.random.choice(X.shape[0], s, replace=False)
    C = X[indices, :]
    R = X[:, indices]

    # Compute SVD of concatenated matrices
    U, sigma, Vt = svds(np.hstack((C.T, R)), k=s)

    # Extract the low-rank representation
    W = np.dot(U[:, :s], np.diag(sigma[:s]))

    # Calculate the residual error
    reconstructed_X = np.dot(W, Vt[:s, :])


    X = X.reshape(-1)  # [1, 2, 3, 4, 5, 6]
    reconstructed_X = reconstructed_X.reshape(-1)
    max_size = max(len(X), len(reconstructed_X))
    X_padded = np.pad(X, (0, max_size - len(X)), mode='constant')
    reconstructed_X_padded = np.pad(reconstructed_X, (0, max_size - len(reconstructed_X)), mode='constant')
    residual_error = X_padded - reconstructed_X_padded


    return W, residual_error

def evolutionary_computation(W, residual_error, num_generations):
    """
    Perform evolutionary computation using a genetic algorithm.

    Parameters:
        W (numpy.ndarray): The low-rank representation.
        residual_error (numpy.ndarray): The residual error.
        num_generations (int): The number of generations.

    Returns:
        final_solution (numpy.ndarray): The final solution found by the genetic algorithm.
    """
    # Define the fitness function

    def evaluate(individual):
        global W
        individual = individual.reshape(-1)
        W = W.reshape(-1)
        max_size = max(len(W), len(individual))
        W_padded = np.pad(W, (0, max_size - len(W)), mode='constant')
        individual = np.pad(individual, (0, max_size - len(individual)), mode='constant')
        new_residual_error = W_padded.dot(individual)

        return np.sum(np.square(new_residual_error - residual_error))

    # Define genetic algorithm parameters
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=W.shape[1])
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate)

    # Initialize population
    population = toolbox.population(n=100)

    # Perform evolution
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=num_generations, verbose=False)

    # Get the best individual
    final_solution = tools.selBest(population, k=1)[0]

    return final_solution

def main(X, s, num_generations):
    """
    Main function to combine the steps of learning low-rank representation
    and performing evolutionary computation.

    Parameters:
        X (numpy.ndarray): The dataset.
        s (int): The number of rows/columns to sample.
        num_generations (int): The number of generations for evolutionary computation.

    Returns:
        final_solution (numpy.ndarray): The final solution found by the evolutionary computation.
    """
    # Step 1: Learning a low-rank representation
    W, residual_error = learn_low_rank_representation(X, s)

    # Step 2: Performing evolutionary computation
    final_solution = evolutionary_computation(W, residual_error, num_generations)

    return final_solution

# Example usage:
X = np.random.rand(10, 10)
s = 4
num_generations = 5
result = main(X, s, num_generations)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


TypeError: object of type 'numpy.float64' has no len()

In [117]:
X = np.random.rand(5, 5)
s = 2
indices = np.random.choice(X.shape[0], s, replace=False)
C = X[indices, :]
R = X[:, indices]
print(C.T.shape, R.shape)
# np.hstack((C.T, R))


# Compute SVD of concatenated matrices
U, sigma, Vt = svds(np.hstack((C.T, R)), k=s)

# Extract the low-rank representation
W = np.dot(U[:, :s], np.diag(sigma[:s]))

# Calculate the residual error
reconstructed_X = np.dot(W, Vt[:s, :])


X = X.reshape(-1)  # [1, 2, 3, 4, 5, 6]
reconstructed_X = reconstructed_X.reshape(-1)
max_size = max(len(X), len(reconstructed_X))
X_padded = np.pad(X, (0, max_size - len(X)), mode='constant')
reconstructed_X_padded = np.pad(reconstructed_X, (0, max_size - len(reconstructed_X)), mode='constant')
residual_error = X_padded - reconstructed_X_padded

# residual_error = X - reconstructed_X

(5, 2) (5, 2)
[ 1.09791500e-01  7.58166117e-01 -6.68470989e-01 -2.17318588e-01
 -7.85734899e-01 -6.87348919e-01  3.52584092e-01 -2.77870978e-01
  5.11033630e-01 -5.96118353e-04  1.43105334e-01 -3.19882899e-03
 -6.50097445e-01 -2.67399903e-01  1.84603734e-01  1.90549382e-01
 -8.78687888e-02  6.38769826e-01 -6.23525595e-02  5.71467399e-01
  1.08779601e-01  6.67099928e-01  5.48732320e-01  1.23377988e-01
  3.44422878e-01]


In [118]:
X = X.reshape(-1)

In [103]:
reconstructed_X

array([[0.11477243, 0.08541815, 0.08674438, 0.55887401],
       [0.95191706, 0.84430115, 0.93067731, 0.234488  ],
       [0.67988977, 0.57935281, 0.62790949, 0.93440875],
       [0.49992143, 0.42947443, 0.46710765, 0.57441104],
       [0.60054623, 0.5287878 , 0.58113598, 0.27316113]])

In [96]:
U[:, :s]

array([[-0.48741081,  0.33130582, -0.24909498, -0.47573955],
       [-0.00914963,  0.45650982, -0.23657013, -0.46396237],
       [ 0.8090027 ,  0.15234246,  0.19402966, -0.43819538],
       [-0.32550018, -0.23464699,  0.80155106, -0.42196132],
       [ 0.04371776, -0.77689626, -0.44928237, -0.43398647]])

In [91]:
sigma

array([0.38681849, 0.49880456, 0.86881056, 2.56567607])

In [93]:
Vt

array([[-0.11190819, -0.33949625,  0.23331042,  0.44217891, -0.56363464,
         0.55188667],
       [-0.34346635, -0.38470642,  0.4171964 ,  0.45051998,  0.35666576,
        -0.47937489],
       [-0.54401516,  0.34082748,  0.59071208, -0.46305588, -0.09183635,
         0.12684128],
       [-0.31688102, -0.43848544, -0.31794718, -0.37188572, -0.52532784,
        -0.43813035]])